In [5]:
rm(list = ls())


In [6]:
library(readxl)   # read Excel supplementary data
library(dplyr)    # data manipulation
library(tidyr)    # reshaping data
library(ggplot2)  # plotting


In [ ]:
# --- MUNRO ET AL. REPRODUCTION (FINAL FIX) ---

# 1. Install & Load Tidyverse
if (!require("tidyverse")) install.packages("tidyverse")
if (!require("viridis")) install.packages("viridis")
if (!require("readxl")) install.packages("readxl") # Ensure readxl is installed
library(tidyverse)
library(viridis)
library(readxl) # Load readxl

# 2. Load the CORRECT Data File
# Note: We removed " info" from the filename to get the actual numbers.
filename <- "41467_2024_54840_MOESM5_ESM.xlsx" # Corrected filename to .xlsx
df <- read_excel(filename, sheet = "Supp Data 2", skip = 17) # Use read_excel and specify sheet and skip if needed

# Define old and new column names for renaming
# These are based on the glimpse output and the data dictionary provided in the Excel file
old_col_names <- c(
  "ESPGEJ",
  "Esophagus_Gastroesophageal_Junction",
  "10857", # Removed backticks
  "5333",  # Removed backticks
  "2400",  # Removed backticks
  "2389",  # Removed backticks
  "970",   # Removed backticks
  "1276",  # Removed backticks
  "1333",  # Removed backticks
  "6215",  # Removed backticks
  "2887",  # Removed backticks
  "3218",  # Removed backticks
  "1064",  # Removed backticks
  "1514",  # Removed backticks
  "1436"   # Removed backticks
)

new_col_names <- c(
  "tissue",
  "tissue_name",
  "genes_total",
  "genes_expression",
  "genes_splicing",
  "genes_isoforms",
  "genes_alt_TSS",
  "genes_alt_polyA",
  "genes_stability",
  "xQTLs_total",
  "xQTLs_eQTL",
  "xQTLs_sQTL",
  "xQTLs_isoQTL",
  "xQTLs_alt_TSS",
  "xQTLs_alt_polyA" # Assuming 'xQTLs_stability' is the one missing, based on 15 cols
)

# Rename the columns
df <- df %>%
  rename_with(~ new_col_names, all_of(old_col_names))

# Check: If this prints 49 rows, you have the right file!
print(paste("Rows loaded:", nrow(df)))

# Inspect the dataframe structure to debug column names
glimpse(df)

# 3. Data Processing
# Filter for Top 15 tissues by total genes
top_tissues <- df %>%
  arrange(desc(genes_total)) %>%
  head(15) %>%
  pull(tissue_name)

# Pivot data to "Long" format for plotting
plot_data <- df %>%
  filter(tissue_name %in% top_tissues) %>%
  select(tissue_name, genes_expression, genes_splicing, genes_isoforms,
         genes_alt_TSS, genes_alt_polyA, genes_stability) %>%
  pivot_longer(cols = -tissue_name, names_to = "Modality", values_to = "Count") %>%
  mutate(Modality = str_to_title(str_remove(Modality, "genes_")))

# 4. Generate & Display Plot
options(repr.plot.width = 12, repr.plot.height = 8) # Set size for Colab

p <- ggplot(plot_data, aes(x = Count, y = reorder(tissue_name, Count), fill = Modality)) +
  geom_bar(stat = "identity", position = "stack", width = 0.8) +
  scale_fill_viridis_d(option = "magma", direction = -1) +
  labs(
    title = "Multimodal RNA Analysis Powers Genetic Discovery",
    subtitle = "Top 15 Tissues (Re-analysis of Munro et al., 2024)",
    x = "Number of Significant Genes",
    y = NULL,
    caption = "Data Source: Nature Communications (2024)"
  ) +
  theme_minimal(base_size = 15) +
  theme(
    legend.position = "bottom",
    panel.grid.major.y = element_blank()
  )

print(p)